In [1]:
from typing import Any, OrderedDict, List, Dict
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision.models.resnet import resnet50
from torchvision.transforms import v2 as T
import matplotlib.pyplot as plt
import json
from torchmetrics import Accuracy
from lightly.transforms.utils import IMAGENET_NORMALIZE
from PIL import Image

## Load Data

In [2]:
class ImageDataset(Dataset):
    def __init__(self, set_map: List[Dict], transform=None) -> None:
        ''' Each item in set_map is expected to contain:
                img_path: Full path to image,
                label: Label corresponding to image at img_path
        '''

        self.set_map = set_map
        self.transform=transform

    def __len__(self):
        return len(self.set_map)
    
    def __getitem__(self, index):   
        sample = self.set_map[index]

        # image = read_image(sample['img_path'])
        image = Image.open(sample['img_path'])

        if self.transform:
            image = self.transform(image)

        return dict(image=image, **sample)

In [3]:
with open('./data/train_unbalanced.json', 'r') as file:
    train_set_map = json.load(file)

In [4]:
with open('./data/test_unbalanced.json', 'r') as file:
    test_set_map = json.load(file)

In [5]:
label_map = {
    'DG': 0,
    'PH': 1,
    'EH': 2,
    'G': 3
}

domain_map = {
    'cartoon': 0,
    'art_painting': 1,
    'photo': 2,
}

In [6]:
for elem in train_set_map:
    elem['label'] = label_map[elem['label']]
for elem in test_set_map:
    elem['label'] = label_map[elem['label']]

In [7]:
transform = T.Compose([
    T.Resize(96),
    T.ToTensor(),
    T.Normalize(
        mean=IMAGENET_NORMALIZE["mean"],
        std=IMAGENET_NORMALIZE["std"],
    ),
])

/home/yasin/miniforge3/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [8]:
train_set = ImageDataset(train_set_map, transform=transform)
test_set = ImageDataset(test_set_map, transform=transform)

In [9]:
train_loader = DataLoader(train_set, batch_size=256, shuffle=True, num_workers=4, persistent_workers=True, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=256, shuffle=False, num_workers=4, persistent_workers=True, pin_memory=True)

## Validate Data

In [10]:
stats = {}
for elem in train_set_map:
    domain = elem['domain']
    label = elem['label']

    if domain not in stats.keys() :
        stats[domain] = {}

    if label not in stats[domain].keys():
        stats[domain][label] = 1
    else:
        stats[domain][label] += 1

In [11]:
stats

{'sketch': {3: 550, 2: 55, 0: 55, 1: 55},
 'photo': {1: 647, 2: 65, 0: 65, 3: 65},
 'cartoon': {2: 703, 1: 70, 3: 70, 0: 70},
 'art_painting': {0: 585, 3: 58, 1: 58, 2: 58}}

In [12]:
stats = {}
for elem in test_set_map:
    domain = elem['domain']
    label = elem['label']

    if domain not in stats.keys() :
        stats[domain] = {}

    if label not in stats[domain].keys():
        stats[domain][label] = 1
    else:
        stats[domain][label] += 1

In [13]:
stats

{'art_painting': {1: 74, 2: 48, 0: 79, 3: 16},
 'sketch': {0: 144, 3: 58, 2: 145, 1: 19},
 'photo': {2: 37, 1: 65, 0: 47, 3: 19},
 'cartoon': {0: 83, 2: 78, 3: 12, 1: 76}}

## Train Models

In [14]:
import pytorch_lightning as L

In [15]:
from typing import Any


class LinearProbe(L.LightningModule):
    def __init__(self, backbone, emb_dim, num_classes, lr=1e-2) -> None:
        super().__init__()
        
        self.backbone: nn.Module = backbone
        for param in self.backbone.parameters():
            param.requires_grad = False

        self.linear_head: nn.Module = nn.Linear(emb_dim, num_classes)
        self.criterion = nn.CrossEntropyLoss()
        self.lr = lr

        self.test_accuracy = Accuracy(task='multiclass', num_classes=num_classes)

    def forward(self, x):
        x = self.backbone(x)
        x = self.linear_head(x)

        return x

    def training_step(self, batch, bacth_idx):
        X = batch['image']
        t = batch['label']

        y = self.forward(X)
        loss = self.criterion(y, t)

        print(loss.item())

        return loss
    
    def test_step(self, batch, batch_idx):
        X = batch['image']
        t = batch['label']

        y = self.forward(X)
        acc = self.test_accuracy(y, t)

        self.log('accuracy', acc, on_epoch=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.linear_head.parameters(), lr=self.lr)

In [16]:
def get_backbone_from_ckpt(ckpt_path: str) -> torch.nn.Module:
    state_dict = torch.load(ckpt_path)["state_dict"]
    state_dict = OrderedDict([
        (".".join(name.split(".")[1:]), param) for name, param in state_dict.items() if name.startswith("backbone")
    ])

    return state_dict

In [17]:
# Baseline Model
model_bl  = resnet50()
weights_bl = get_backbone_from_ckpt("./data/r50_bl.ckpt")
model_bl.load_state_dict(weights_bl, strict=False)
model_bl.fc = torch.nn.Identity()
model_bl = model_bl.cuda()

In [19]:
# MixStyle Model
model_ms  = resnet50()
weights_ms = get_backbone_from_ckpt("./data/r50_ms.ckpt")
model_ms.load_state_dict(weights_ms, strict=False)
model_ms.fc = torch.nn.Identity()
model_ms = model_ms.cuda()

In [20]:
baseline_module = LinearProbe(model_bl, emb_dim=2048, num_classes=4)

In [21]:
mixstyle_module = LinearProbe(model_ms, emb_dim=2048, num_classes=4)

In [22]:
trainer = L.Trainer(
    max_epochs=100
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [23]:
trainer.fit(baseline_module, train_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Params
-----------------------------------------------------
0 | backbone      | ResNet             | 23.5 M
1 | linear_head   | Linear             | 8.2 K 
2 | criterion     | CrossEntropyLoss   | 0     
3 | test_accuracy | MulticlassAccuracy | 0     
-----------------------------------------------------
8.2 K     Trainable params
23.5 M    Non-trainable params
23.5 M    Total params
94.065    Total estimated model params size (MB)
/home/yasin/miniforge3/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (13) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

1.4451196193695068
0.47646602988243103
0.23262853920459747
0.1957593411207199
0.2146281599998474
0.20582640171051025
0.21871455013751984
0.29860013723373413
0.19668836891651154
0.31016862392425537
0.3174445331096649
0.2262854427099228
0.15725505352020264
0.10438891500234604
0.08279526233673096
0.07885938137769699
0.12504485249519348
0.11277221888303757
0.14849573373794556
0.15097539126873016
0.19396379590034485
0.2115233689546585
0.11977659165859222
0.07348444312810898
0.13225358724594116
0.3303788900375366
0.0895947739481926
0.09898501634597778
0.19428136944770813
0.06601384282112122
0.08298753947019577
0.10005137324333191
0.057477276772260666
0.07260476797819138
0.11884951591491699
0.13897714018821716
0.0964621752500534
0.11011116206645966
0.08749894797801971
0.10799213498830795
0.036918532103300095
0.04928474500775337
0.10206928849220276
0.05097725987434387
0.06938038021326065
0.11141134798526764
0.06183546409010887
0.04286559671163559
0.06587182730436325
0.04360438138246536
0.09630

`Trainer.fit` stopped: `max_epochs=100` reached.


In [24]:
trainer.test(baseline_module, dataloaders=train_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/yasin/miniforge3/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Testing: |          | 0/? [00:00<?, ?it/s]

/home/yasin/miniforge3/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 256. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/home/yasin/miniforge3/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 157. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.9981418251991272     │
└───────────────────────────┴───────────────────────────┘

[{'accuracy': 0.9981418251991272}]

In [25]:
trainer.test(baseline_module, dataloaders=test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

/home/yasin/miniforge3/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 232. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.8350000381469727     │
└───────────────────────────┴───────────────────────────┘

[{'accuracy': 0.8350000381469727}]

In [26]:
trainer = L.Trainer(
    max_epochs=100
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [27]:
trainer.fit(mixstyle_module, train_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Params
-----------------------------------------------------
0 | backbone      | ResNet             | 23.5 M
1 | linear_head   | Linear             | 8.2 K 
2 | criterion     | CrossEntropyLoss   | 0     
3 | test_accuracy | MulticlassAccuracy | 0     
-----------------------------------------------------
8.2 K     Trainable params
23.5 M    Non-trainable params
23.5 M    Total params
94.065    Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

1.5087896585464478
0.3351497948169708
0.2200234830379486
0.26780545711517334
0.22290301322937012
0.21519321203231812
0.2054130584001541
0.15842141211032867
0.3941017687320709
0.1551031619310379
0.25226256251335144
0.29123246669769287
0.10914649814367294
0.1946232169866562
0.1303073763847351
0.18579453229904175
0.08382158726453781
0.14693768322467804
0.2926125228404999
0.14106059074401855
0.20733974874019623
0.08218329399824142
0.1428484320640564
0.13272906839847565
0.11869131773710251
0.17994222044944763
0.09603496640920639
0.1286541074514389
0.08606452494859695
0.07122112065553665
0.17118602991104126
0.11354679614305496
0.10579408705234528
0.10604926943778992
0.0734524130821228
0.09228632599115372
0.07145953923463821
0.08346434682607651
0.0758248120546341
0.032836444675922394
0.0582764558494091
0.05932967737317085
0.10342191904783249
0.09126985818147659
0.04651588574051857
0.10544690489768982
0.11899767071008682
0.07520515471696854
0.0893748328089714
0.07231893390417099
0.087075419723

`Trainer.fit` stopped: `max_epochs=100` reached.


0.0051596928387880325


In [28]:
trainer.test(mixstyle_module, dataloaders=train_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │            1.0            │
└───────────────────────────┴───────────────────────────┘

[{'accuracy': 1.0}]

In [29]:
trainer.test(mixstyle_module, dataloaders=test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.8630000352859497     │
└───────────────────────────┴───────────────────────────┘

[{'accuracy': 0.8630000352859497}]